In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.datasets import cifar10
from skimage.transform import resize
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
import pickle
from scipy.stats import truncnorm
# modules
from Ops.spectral_normalization import SpectralConv2D, SpectralDense
from Ops.ops import ResnetBlock, ResnetBlockUp, ResnetBlockDown
from Ops.attention import Attention
from Ops.global_sum_pooling import GlobalSumPooling2D
from Ops.conditional_batch_normalization import ConditionalBatchNormalization

In [2]:
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)

In [3]:
# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
    # load inception v3 model
    model = InceptionV3()
    # enumerate splits of images/predictions
    scores = list()
    n_part = math.floor(images.shape[0] / n_split)
    for i in range(n_split):
        # retrieve images
        ix_start, ix_end = i * n_part, (i+1) * n_part
        subset = images[ix_start:ix_end]
        # convert from uint8 to float32
        subset = subset.astype('float32')
        # scale images to the required size
        subset = scale_images(subset, (299,299,3))
        # pre-process images, scale to [-1,1]
        subset = preprocess_input(subset)
        # predict p(y|x)
        p_yx = model.predict(subset)
        # calculate p(y)
        p_y = np.expand_dims(p_yx.mean(axis=0), 0)
        # calculate KL divergence using log probabilities
        kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
        # sum over classes
        sum_kl_d = kl_d.sum(axis=1)
        # average over images
        avg_kl_d = np.mean(sum_kl_d)
        # undo the log
        is_score = np.exp(avg_kl_d)
        # store
        scores.append(is_score)
    # average across images
    is_avg, is_std = np.mean(scores), np.std(scores)
    return is_avg, is_std

In [4]:
# calculate frechet inception distance
def calculate_fid(images1, images2):
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
    # convert integer to floating point values
    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    # resize images
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [5]:
# generate points in latent space as input for the generator
def generate_latent_points(n_samples, latent_dim=100, n_classes=10, bigGAN=False):
    if not bigGAN:
        # generate points in the latent space
        z_input = np.random.randn(n_samples * latent_dim)
        # reshape into a batch of inputs for the network
        z_input = z_input.reshape(n_samples, latent_dim)
    else:
        z_input = truncnorm.rvs(-2, 2, size=(n_samples, latent_dim), random_state=None).astype(np.float32)
        z_input = z_input * 2.0 # 2.0 is truncation value
    # generate labels
    labels = np.random.randint(0, n_classes, n_samples)
    return [z_input, labels]

In [6]:
def calc_is_for_gan(generator, n_split=10, eps=1E-16, custom_objects=None, latent_dim=100):
    generator = tf.keras.models.load_model(generator, custom_objects=custom_objects)
    #generate latent points for 50k images
    [z_input, labels] = generate_latent_points(25000, latent_dim=latent_dim)
    images = generator.predict([z_input, labels])
    images = (images*127.5) + 127.5
    print('generated ', images.shape)
    #calculate IS score
    is_avg, is_std = calculate_inception_score(images, n_split=n_split)
    print('score', is_avg, is_std)

In [7]:
def calc_fid_for_gan(generator, custom_objects=None, latent_dim=100):
    generator = tf.keras.models.load_model(generator, custom_objects=custom_objects)
    #generate latent points for 50k images
    [z_input, labels] = generate_latent_points(10000, latent_dim=latent_dim)
    fake_images = generator.predict([z_input, labels])
    fake_images = (fake_images*127.5) + 127.5
    print('generated ', fake_images.shape)
    (real_images, _), (_, _) = cifar10.load_data()
    np.random.shuffle(real_images)
    real_images = real_images[:10000]
    #calculate FID score
    fid = calculate_fid(real_images, fake_images)
    print('FID', fid)

In [8]:
def calc_is_for_bigGAN(generator, n_split=10, eps=1E-16, custom_objects=None):
    generator = tf.keras.models.load_model(generator, custom_objects=custom_objects)
    #generate latent points for 50k images
    [z_input, labels] = generate_latent_points(25000, latent_dim=128, bigGAN=True)
    images = generator.predict([z_input, labels])
    images = (images*127.5) + 127.5
    print('generated ', images.shape)
    #calculate IS score
    is_avg, is_std = calculate_inception_score(images)
    print('score', is_avg, is_std)

In [9]:
def calc_fid_for_bigGAN(generator, custom_objects=None):
    generator = tf.keras.models.load_model(generator, custom_objects=custom_objects)
    #generate latent points for 50k images
    [z_input, labels] = generate_latent_points(10000, latent_dim=128, bigGAN=True)
    fake_images = generator.predict([z_input, labels])
    fake_images = (fake_images*127.5) + 127.5
    print('generated ', fake_images.shape)
    (real_images, _), (_, _) = cifar10.load_data()
    np.random.shuffle(real_images)
    real_images = real_images[:10000]
    #calculate FID score
    fid = calculate_fid(real_images, fake_images)
    print('FID', fid)

# Evaluate Models

## AC-GAN_CIFAR10_1 (Base model)
<p>Batch Size : 64</p>
<p>D steps per G step = 1</p>

In [21]:
#measure IS
generator = './history/acgan/acgan-cifar10-1/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator, n_split=5)

In [ ]:
#measure FID
generator = './history/acgan/acgan-cifar10-1/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

In [10]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-1/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, [np.ones((10000,1)),testY])
print(testX.shape[0])

313/313 [==============================] - 1s 3ms/step - loss: 1.2250 - out_fake_loss: 0.3515 - out_aux_loss: 0.8735 - out_fake_accuracy: 0.8516 - out_aux_out_aux_sparse_categorical_accuracy: 0.7696
10000


## AC-GAN_CIFAR10_2 (Base model->Smaller Batch Size)
<p>Batch Size : 32</p>
<p>D steps per G step = 1</p>

In [ ]:
#measure IS
generator = './history/acgan/acgan-cifar10-2/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator)

In [ ]:
#measure FID
generator = './history/acgan/acgan-cifar10-2/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

In [16]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-2/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 0s 3ms/step - loss: -2.0618 - out_fake_loss: -3.0276 - out_aux_loss: 0.9658 - out_fake_accuracy: 0.1098 - out_aux_out_aux_sparse_categorical_accuracy: 0.7614


## AC-GAN_CIFAR10_3 (Base model -> Bigger Batch Size)
<p>Batch Size : 128</p>
<p>D steps per G step = 1</p>

In [ ]:
#measure IS
generator = './history/acgan/acgan-cifar10-3/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator)

In [ ]:
#measure FID
generator = './history/acgan/acgan-cifar10-3/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

In [ ]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-3/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

## AC-GAN_CIFAR10_4 (Base model -> 50% fmap increase in G)
<p>Batch Size : 64</p>
<p>D steps per G step = 1</p>
<p>G: all num_filters increased by 50%</p>

In [ ]:
#measure IS
generator = './history/acgan/acgan-cifar10-4/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator)

In [ ]:
#measure FID
generator = './history/acgan/acgan-cifar10-4/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

In [17]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-4/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 0s 3ms/step - loss: -3.1748 - out_fake_loss: -4.1654 - out_aux_loss: 0.9906 - out_fake_accuracy: 0.0993 - out_aux_out_aux_sparse_categorical_accuracy: 0.7574


## AC-GAN_CIFAR10_11 (Base model) Disc. LR Halved
<p>Batch Size : 64</p>
<p>D steps per G step = 1</p>

In [17]:
#measure IS
generator = './history/acgan/acgan-cifar10-11/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator, n_split=5)

generated  (25000, 32, 32, 3)
score 5.745974 0.060925893


In [15]:
#measure FID
generator = './history/acgan/acgan-cifar10-11/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

generated  (10000, 32, 32, 3)
FID 50.32705359927204


In [11]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-11/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 1s 3ms/step - loss: 2.1301 - out_fake_loss: 1.4816 - out_aux_loss: 0.6485 - out_fake_accuracy: 0.1010 - out_aux_out_aux_sparse_categorical_accuracy: 0.7896


## AC-GAN_CIFAR10_12 (Base model->Smaller Batch Size) Disc. LR Halved
<p>Batch Size : 32</p>
<p>D steps per G step = 1</p>

In [22]:
#measure IS
generator = './history/acgan/acgan-cifar10-12/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator, latent_dim=110)

generated  (25000, 32, 32, 3)
score 5.8037267 0.03910174


In [23]:
#measure FID
generator = './history/acgan/acgan-cifar10-12/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator, latent_dim=110)

generated  (10000, 32, 32, 3)
FID 48.727771455353235


In [12]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-12/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 1s 4ms/step - loss: 2.1496 - out_fake_loss: 1.5044 - out_aux_loss: 0.6452 - out_fake_accuracy: 0.1087 - out_aux_out_aux_sparse_categorical_accuracy: 0.7932


## AC-GAN_CIFAR10_13 (Base model -> Bigger Batch Size) Disc. LR Halved
<p>Batch Size : 128</p>
<p>D steps per G step = 1</p>

In [28]:
#measure IS
generator = './history/acgan/acgan-cifar10-13/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator, latent_dim=110)

generated  (25000, 32, 32, 3)
score 3.4769855 0.041540418


In [29]:
#measure FID
generator = './history/acgan/acgan-cifar10-13/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator, latent_dim=110)

generated  (10000, 32, 32, 3)
FID 126.96982927891284


In [27]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-13/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 0s 3ms/step - loss: -18.8897 - out_fake_loss: -19.5395 - out_aux_loss: 0.6498 - out_fake_accuracy: 0.0971 - out_aux_out_aux_sparse_categorical_accuracy: 0.7802


## AC-GAN_CIFAR10_14 (Base model -> 50% fmap increase in G) Disc. LR Halved
<p>Batch Size : 64</p>
<p>D steps per G step = 1</p>
<p>G: all num_filters increased by 50%</p>

In [25]:
#measure IS
generator = './history/acgan/acgan-cifar10-14/training_checkpoints/generator-e250.h5'
calc_is_for_gan(generator)

generated  (25000, 32, 32, 3)
score 6.028472 0.07863616


In [26]:
#measure FID
generator = './history/acgan/acgan-cifar10-14/training_checkpoints/generator-e250.h5'
calc_fid_for_gan(generator)

generated  (10000, 32, 32, 3)
FID 49.622968049923614


In [13]:
#measure accuracy on CIFAR10 test data
discriminator = './history/acgan/acgan-cifar10-14/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 1s 3ms/step - loss: 1.7978 - out_fake_loss: 1.1275 - out_aux_loss: 0.6703 - out_fake_accuracy: 0.1033 - out_aux_out_aux_sparse_categorical_accuracy: 0.7890


# AC-BigGAN_CIFAR10_1
<p>Batch Size : 64 </p>
<p>D steps per G step = 1</p>

In [10]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-1/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.7079735 0.13069068


In [11]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-1/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 29.91962002982266


In [12]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-1/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 12ms/step - loss: -7.1989 - out_fake_loss: -7.6671 - out_aux_loss: 0.4683 - out_fake_accuracy: 0.1340 - out_aux_out_aux_sparse_categorical_accuracy: 0.8539


# AC-BigGAN_CIFAR10_2

In [10]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-2/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.720698 0.1293581


In [11]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-2/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 25.329377660606


In [12]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-2/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 12ms/step - loss: -4.3929 - out_fake_loss: -4.7763 - out_aux_loss: 0.3834 - out_fake_accuracy: 0.1312 - out_aux_out_aux_sparse_categorical_accuracy: 0.8838


# AC-BigGAN_CIFAR10_3

In [13]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-3/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.7376604 0.092479035


In [14]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-3/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 26.802800134554204


In [15]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-3/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 11ms/step - loss: -3.3180 - out_fake_loss: -3.7502 - out_aux_loss: 0.4322 - out_fake_accuracy: 0.1361 - out_aux_out_aux_sparse_categorical_accuracy: 0.8652


# AC-BigGAN_CIFAR10_4

In [11]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-4/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.72899 0.17522876


In [12]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-4/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 27.963016764337056


In [10]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-4/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 12ms/step - loss: -5.8492 - out_fake_loss: -6.2778 - out_aux_loss: 0.4286 - out_fake_accuracy: 0.1447 - out_aux_out_aux_sparse_categorical_accuracy: 0.8683


# AC-BigGAN_CIFAR10_5

In [10]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-5/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.646039 0.09679922


In [11]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-5/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 25.809598875721314


In [12]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-5/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 12ms/step - loss: -3.0351 - out_fake_loss: -3.5048 - out_aux_loss: 0.4697 - out_fake_accuracy: 0.1328 - out_aux_out_aux_sparse_categorical_accuracy: 0.8453


# AC-BigGAN_CIFAR10_6

In [15]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-6/training_checkpoints/generator-e250.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 7.814658 0.10833879


In [14]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-6/training_checkpoints/generator-e250.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 23.976878205521324


In [13]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-6/training_checkpoints/discriminator-e250.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 2s 11ms/step - loss: -1.7444 - out_fake_loss: -2.1399 - out_aux_loss: 0.3956 - out_fake_accuracy: 0.1342 - out_aux_out_aux_sparse_categorical_accuracy: 0.8791


# AC-BigGAN_CIFAR10_9
<p>Batch Size : 64 </p>
<p>D steps per G step = 1</p>
"Mistake" fix

In [10]:
#measure IS
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-9/training_checkpoints/generator-e200.h5'
calc_is_for_bigGAN(generator, custom_objects=custom_objects)

generated  (25000, 32, 32, 3)
score 6.917647 0.14044906


In [11]:
#measure FID
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
generator = './history/bigacgan/bigacgan-cifar10-9/training_checkpoints/generator-e200.h5'
calc_fid_for_bigGAN(generator, custom_objects=custom_objects)

generated  (10000, 32, 32, 3)
FID 30.1998249705484


In [15]:
#measure accuracy on CIFAR10 test data
custom_objects={'SpectralDense': SpectralDense, 'SpectralConv2D': SpectralConv2D}
discriminator = './history/bigacgan/bigacgan-cifar10-9/training_checkpoints/discriminator-e200.h5'
discriminator = tf.keras.models.load_model(discriminator, custom_objects=custom_objects)
(_, _), (testX, testY) = cifar10.load_data()
testX = testX.astype('float32')
testX = (testX - 127.5) / 127.5
_eval = discriminator.evaluate(testX, testY, batch_size=64)

157/157 [==============================] - 1s 9ms/step - loss: -2.0863 - out_fake_loss: -2.7290 - out_aux_loss: 0.6427 - out_fake_accuracy: 0.1226 - out_aux_out_aux_sparse_categorical_accuracy: 0.8112
